# Konfiguracja

In [7]:
# Algorytmy Grafowe
# Piotr Faliszewski 2019
# Load graph in the DIMACS ascii format + weights

def loadCNFFormula( name ): # Load a CNF formula in the DIMACS ascii format from the file "name" and return it as a list of clauses, 
    V = 0                   # Returns (V,F), V -- highest variable number F -- list of clauses
    L = []  
    f = open( name, "r" )
    lines = f.readlines()
    for l in lines:
        s = l.split()
        if(len(s) < 1): continue
        if( s[0] == "c" ):
            print(s)
            continue
        elif( s[0] == "p" ):
            V = int(s[2])
        else:
            clause = [int(v) for v in s[:-1]]
            L.append(clause)
    f.close()
    return (V,L)

def loadWeightedGraph( name ): # Load a graph in the DIMACS ascii format (with weights) from the file "name" and return it as a list of sets
    V = 0                      # Returns (V,L), V -- number of vertices (1, ..., V), L -- list of edges in the format (x,y,w): edge between x and y with weight w (x<y)"""
    L = []  
    f = open( name, "r" )
    lines = f.readlines()
    for l in lines:
        s = l.split()
        if(len(s) < 1): continue
        if( s[0] == "c" ):
            continue
        elif( s[0] == "p" ):
            V = int(s[2])
        elif( s[0] == "e" ):
                (a,b,c) = (int(s[1]), int(s[2]), int(s[3]))
                (x,y,c) = (min(a,b), max(a,b), c)
                L.append((x,y,c))
    f.close()
    return (V,L)

def loadDirectedWeightedGraph( name ): # Load a directed graph in the DIMACS ascii format (with weights) from the file "name" and return it as a list of sets
    V = 0                              # Returns (V,L), V -- number of vertices (1, ..., V), L -- list of edges in the format (x,y,w): edge between x and y with weight w
    L = []
    f = open( name, "r" )
    lines = f.readlines()
    for l in lines:
        s = l.split()
        if(len(s) < 1): 
            continue
        if( s[0] == "c" ):
            continue
        elif( s[0] == "p" ):
            V = int(s[2])
        elif( s[0] == "e" ):
            (a,b,c) = (int(s[1]), int(s[2]), int(s[3]))
            L.append((a,b,c))
    f.close()
    return (V,L)

def readSolution(name): # Read the expected solution from the first line of the graph file
    with open(name, 'r') as f:
        line = f.readline()
        return line.split()[-1]

In [8]:
import networkx as nx
import matplotlib.pyplot as plt

def show(x): # ((V,L))
    g = nx.DiGraph()
    g.add_weighted_edges_from(x[1])
    pos = nx.spring_layout(g)
    labels = nx.get_edge_attributes(g,'weight')
    nx.draw_networkx_edge_labels(g,pos,edge_labels=labels,font_size=6,rotate=False)
    nx.draw_networkx_nodes(g, pos, node_size=150, node_color='red', alpha=0.6)
    nx.draw_networkx_edges(g, pos, width=0.2, arrowsize=4, alpha=0.9)
    nx.draw_networkx_labels(g, pos, font_size=7, font_family='sans-serif')
    plt.savefig("graph", dpi=250)

In [10]:
import time
import os

def test():
    l = list(os.listdir('flow/'))
    cnt=0
    for i in range(len(l)):
        V, L = loadDirectedWeightedGraph('flow/'+l[i])
        G=Graph(V,L)
        time0=time.time()
        res=G.max_flow(0,V-1)
        time1=time.time()
        print("="*50)
        print("| #"+str(i+1),"Wynik:",res,"| Oczekiwane:",readSolution('flow/'+l[i]),"| Czas:",round(time1-time0,3))
        if res==int(readSolution('flow/'+l[i])):
            print("| Test zaliczony!"+l[i])
            cnt+=1
        else:
            print("| TEST NIEZALICZONY!   "+l[i])
    print("="*50)
    print("Ilosc zaliczonych testów:", str(cnt)+"/"+str(len(l))) 

# Rozwiązanie

### Klasa wierzchołka

In [11]:
class Node:
    def __init__(self):
        self.edges = {}
    
    def add_edge(self, to, weight):
        self.edges[to] = self.edges.get(to,0) + weight
    
    def del_edge(self, to):
        del self.edges[to]
    
    def __str__(self):
        return str(list(self.edges.items()))

### Klasa grafu

In [12]:
class Graph:
    def __init__(self,V,L):
        self.V=V
        self.L=L
        self.G=[Node() for i in range(V)]
        for (x,y,c) in L:
            self.G[x-1].add_edge(y-1,c)
            self.G[y-1].add_edge(x-1,0)
            
    def show(self):
        show((self.V,self.L))
            
    def __str__(self):
        res=""
        for v in range(V):
            res+= str(v) + (" -> ") + str(self.G[v]) + "\n"
        return res
    
    def BFS(self,source,sink,parent):
        visited = [False]*(self.V)
        Q=[]
        Q.append(source)
        visited[source] = True
        while Q:
            u = Q.pop()
            for v in self.G[u].edges:
                if not visited[v] and self.G[u].edges[v]>0:
                    Q.append(v)
                    visited[v] = True
                    parent[v] = u
                    if v == sink:
                        return True
        return False 
    
    def max_flow(self,source,sink):
        parent=[-1]*(self.V)
        max_flow=0
        while self.BFS(source,sink,parent): # if augumenting path exists
            path_flow=float("Inf")
            s=sink
            while(s!=source): # find the maximal value of flow in the augumenting path
                path_flow = min(path_flow,self.G[parent[s]].edges.get(s))
                s=parent[s]
            max_flow+=path_flow # update the max flow
            v=sink
            while(v!=source): # update the graph
                u=parent[v]
                self.G[u].edges[v]-=path_flow
                self.G[v].edges[u]+=path_flow
                v=parent[v]
        return max_flow


In [13]:
test()

| #1 Wynik: 4816 | Oczekiwane: 4816 | Czas: 0.035
| Test zaliczony!clique100
| #2 Wynik: 709 | Oczekiwane: 709 | Czas: 0.001
| Test zaliczony!clique20
| #3 Wynik: 138 | Oczekiwane: 138 | Czas: 0.001
| Test zaliczony!clique5
| #4 Wynik: 4179 | Oczekiwane: 4179 | Czas: 0.041
| Test zaliczony!grid100x100
| #5 Wynik: 15 | Oczekiwane: 15 | Czas: 0.0
| Test zaliczony!grid5x5
| #6 Wynik: 124 | Oczekiwane: 124 | Czas: 0.001
| Test zaliczony!pp100
| #7 Wynik: 1574 | Oczekiwane: 1574 | Czas: 0.0
| Test zaliczony!rand100_500
| #8 Wynik: 132 | Oczekiwane: 132 | Czas: 0.0
| Test zaliczony!rand20_100
| #9 Wynik: 23 | Oczekiwane: 23 | Czas: 0.0
| Test zaliczony!simple
| #10 Wynik: 5 | Oczekiwane: 5 | Czas: 0.0
| Test zaliczony!simple2
| #11 Wynik: 3 | Oczekiwane: 3 | Czas: 0.0
| Test zaliczony!trivial
| #12 Wynik: 7 | Oczekiwane: 7 | Czas: 0.0
| Test zaliczony!trivial2
| #13 Wynik: 20 | Oczekiwane: 20 | Czas: 0.0
| Test zaliczony!worstcase
Ilosc zaliczonych testów: 13/13
